In [1]:
import os
from athina.evals import CustomPrompt
from athina.loaders import Loader
from athina.keys import AthinaApiKey, OpenAiApiKey
import pandas as pd

OpenAiApiKey.set_key(os.getenv('OPENAI_API_KEY'))
AthinaApiKey.set_key(os.getenv('ATHINA_API_KEY'))

/Users/akshat_g/athina/repos/athina-evals/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/akshat_g/athina/repos/athina-evals/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Create batch dataset from list of dict objects
raw_data = [
    {
        "query": "What is the capital of Greece?",
        "context": ["Greece is often called the cradle of Western civilization."],
        "response": "Athens",
    },
    {
        "query": "What is the price of a Tesla Model 3?",
        "context": ["Tesla Model 3 is a fully electric car."],
        "response": "I cannot answer this question as prices vary from country to country.",
    },
    {
        "query": "What is a shooting star?",
        "context": ["Black holes are stars that have collapsed under their own gravity. They are so dense that nothing can escape their gravitational pull, not even light."],
        "response": "A shooting star is a meteor that burns up in the atmosphere.",
    }
]

dataset = Loader().load_dict(raw_data)
pd.DataFrame(dataset)

,query,context,response,expected_response
0,What is the capital of Greece?,[Greece is often called the cradle of Western ...,Athens,None
1,What is the price of a Tesla Model 3?,[Tesla Model 3 is a fully electric car.],I cannot answer this question as prices vary f...,None
2,What is a shooting star?,[Black holes are stars that have collapsed und...,A shooting star is a meteor that burns up in t...,None


In [3]:
# Checks if the LLM response answers the user query sufficiently
eval_model = "gpt-3.5-turbo"
eval_prompt = """
If the response refuses to answer the user's query, then fail. Otherwise pass.

User Query: {query}
Response: {response}"""
CustomPrompt(
    eval_prompt=eval_prompt, 
    model=eval_model, 
    display_name="Response should answer user's query",
).run_batch(data=dataset).to_df()

,query,context,response,expected_response,display_name,failed,grade_reason,runtime,model,passed
0,What is the capital of Greece?,[Greece is often called the cradle of Western civilization.],Athens,None,Response should answer user's query,False,The response provided the correct answer to the user's query.,1350,gpt-3.5-turbo,1.0
1,What is the price of a Tesla Model 3?,[Tesla Model 3 is a fully electric car.],I cannot answer this question as prices vary from country to country.,None,Response should answer user's query,True,"The response refuses to answer the user's query, which does not meet the criteria for a pass.",1161,gpt-3.5-turbo,0.0
2,What is a shooting star?,"[Black holes are stars that have collapsed under their own gravity. They are so dense that nothing can escape their gravitational pull, not even light.]",A shooting star is a meteor that burns up in the atmosphere.,None,Response should answer user's query,False,"The response provides a clear answer to the user's query, explaining that a shooting star is a meteor that burns up in the atmosphere.",1232,gpt-3.5-turbo,1.0
